In [4]:
from pymongo import MongoClient

In [5]:
client = MongoClient("mongodb://localhost:27018/")

In [6]:
db = client["mydb"]
coll = db.get_collection(name="employee")

In [7]:
for i in coll.find():
    print(i)

{'_id': ObjectId('67e4d8003149916b3ab7a75b'), 'employee_id': 205, 'first_name': 'Shelley', 'last_name': 'Higgins', 'hire_date': '2002-06-07'}
{'_id': ObjectId('67e4d8003149916b3ab7a75c'), 'employee_id': 204, 'first_name': 'Hermann', 'last_name': 'Baer', 'hire_date': '2002-06-07'}
{'_id': ObjectId('67e4d8003149916b3ab7a75d'), 'employee_id': 203, 'first_name': 'Susan', 'last_name': 'Mavris', 'hire_date': '2002-06-07'}
{'_id': ObjectId('67e4d8003149916b3ab7a75e'), 'employee_id': 202, 'first_name': 'Pat', 'last_name': 'Fay', 'hire_date': '2005-08-17'}
{'_id': ObjectId('67e4d8003149916b3ab7a75f'), 'employee_id': 201, 'first_name': 'Michael', 'last_name': 'Hartstein', 'hire_date': '2004-02-17'}
{'_id': ObjectId('67e4d8003149916b3ab7a760'), 'employee_id': 200, 'first_name': 'Jennifer', 'last_name': 'Whalen', 'hire_date': '2003-09-17'}
{'_id': ObjectId('67e4d8003149916b3ab7a761'), 'employee_id': 199, 'first_name': 'Douglas', 'last_name': 'Grant', 'hire_date': '2008-01-13'}
{'_id': ObjectId('67

In [8]:
employees = db.get_collection(name="employee")


employee_document = {
    "employee_id": 219,
    "first_name": "Hailed",
    "last_name": "Mary",
    "hire_date": "2012-06-07"
}

inserted_id = employees.insert_one(employee_document).inserted_id
print(f"Inserted document ID: {inserted_id}")


Inserted document ID: 680226364fdbe281ead0ccbd


In [9]:
employee = employees.find_one({"employee_id": 219})

print("Employee Details:")
print(employee)


Employee Details:
{'_id': ObjectId('680226364fdbe281ead0ccbd'), 'employee_id': 219, 'first_name': 'Hailed', 'last_name': 'Mary', 'hire_date': '2012-06-07'}


In [10]:
update_result = employees.update_one(
    {"employee_id": 219},  
    {"$set": {"first_name": "Shelley Updated"}}  
)

print(f"Matched {update_result.matched_count} document, Modified {update_result.modified_count} document")


Matched 1 document, Modified 1 document


In [11]:
delete_result = employees.delete_one({"employee_id": 219})

print(f"Deleted {delete_result.deleted_count} document")


Deleted 1 document


In [12]:
employees.create_index([("employee_id", 1)])  # 1 means ascending order

print("Index on 'employee_id' created.")


Index on 'employee_id' created.


In [13]:
from pymongo import DESCENDING
from datetime import datetime

employees.update_many(
    {},
    [{"$set": {"hire_date": {"$dateFromString": {"dateString": "$hire_date"}}}}]
)

pipeline = [
    {
        "$group": {
            "_id": {"$year": "$hire_date"},  # Group by hire year
            "total_employees": {"$sum": 1}
        }
    },
    {"$sort": {"_id": DESCENDING}}  # Sort by year descending
]

result = employees.aggregate(pipeline)

for year in result:
    print(f"Year: {year['_id']}, Total Employees: {year['total_employees']}")

Year: 2009, Total Employees: 1
Year: 2008, Total Employees: 3
Year: 2007, Total Employees: 8
Year: 2006, Total Employees: 8
Year: 2005, Total Employees: 14
Year: 2004, Total Employees: 3
Year: 2003, Total Employees: 5
Year: 2002, Total Employees: 6
Year: 2001, Total Employees: 1


In [ ]:
pipeline = [
    #1. Match employees hired after '2000-01-01'
    {
        "$match": {
            "hire_date": { "$gt": datetime(2000, 1, 1) }
        }
    },
    
    #2. Group by first_name and count the number of employees with that first_name
    {
        "$group": {
            "_id": "$first_name", 
            "total_employees": { "$sum": 1 }  
        }
    },
    
    # 3. Sort the result by the count of employees in descending order
    {
        "$sort": { "total_employees": -1 }
    },
    
    #4. Project the result to show only the first_name and total_employees
    {
        "$project": {
            "_id": 0,  
            "first_name": "$_id",  
            "total_employees": 1  
        }
    }
]

result = employees.aggregate(pipeline)

for doc in result:
    print(doc)

In [14]:
query = {
    "hire_date": {"$gt": datetime(2000, 1, 1)}
}

employees_after_2000 = employees.find(query)

for emp in employees_after_2000:
    print(emp)


{'_id': ObjectId('67e4d8003149916b3ab7a75b'), 'employee_id': 205, 'first_name': 'Shelley', 'last_name': 'Higgins', 'hire_date': datetime.datetime(2002, 6, 7, 0, 0)}
{'_id': ObjectId('67e4d8003149916b3ab7a75c'), 'employee_id': 204, 'first_name': 'Hermann', 'last_name': 'Baer', 'hire_date': datetime.datetime(2002, 6, 7, 0, 0)}
{'_id': ObjectId('67e4d8003149916b3ab7a75d'), 'employee_id': 203, 'first_name': 'Susan', 'last_name': 'Mavris', 'hire_date': datetime.datetime(2002, 6, 7, 0, 0)}
{'_id': ObjectId('67e4d8003149916b3ab7a75e'), 'employee_id': 202, 'first_name': 'Pat', 'last_name': 'Fay', 'hire_date': datetime.datetime(2005, 8, 17, 0, 0)}
{'_id': ObjectId('67e4d8003149916b3ab7a75f'), 'employee_id': 201, 'first_name': 'Michael', 'last_name': 'Hartstein', 'hire_date': datetime.datetime(2004, 2, 17, 0, 0)}
{'_id': ObjectId('67e4d8003149916b3ab7a760'), 'employee_id': 200, 'first_name': 'Jennifer', 'last_name': 'Whalen', 'hire_date': datetime.datetime(2003, 9, 17, 0, 0)}
{'_id': ObjectId('6

In [ ]:
session = client.start_session()

with session.start_transaction():
    try:
        employees.update_one({"employee_id": 205}, {"$set": {"first_name": "Shelley Transaction"}}, session=session)
        employees.update_one({"employee_id": 206}, {"$set": {"first_name": "John Transaction"}}, session=session)

        session.commit_transaction()
        print("Transaction committed!")
    except Exception as e:
        session.abort_transaction()
        print(f"Transaction aborted due to error: {e}")


Transaction aborted due to error: Transaction numbers are only allowed on a replica set member or mongos, full error: {'ok': 0.0, 'errmsg': 'Transaction numbers are only allowed on a replica set member or mongos', 'code': 20, 'codeName': 'IllegalOperation'}
